In [ ]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install torch
import transformers
import torch

In [ ]:
from huggingface_hub import login
login()

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    # The quantization line
    model_kwargs={"torch_dtype": torch.bfloat16, "load_in_4bit": True}
)

In [ ]:
import transformers
import torch

def GetEntity(frase,entita):

  prompt=f"""
        Your task is to generate sentences while leaving the meaning unchanged.
        You will be given as input a sentence and the name of an entity, create a new sentence
        leaving the given entity and general meaning unchanged.
        Do not add comments or explanations. It is not enough to change names of people or places. Try to change words and use synonyms.
        Entità: {entita}
        Frase: {frase}
        Nuova Frase:
  """

  messages = [
    {"role": "system", "content": "Answer questions"},
    {"role": "user", "content": f"{prompt}"},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
  )

  terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
  )


  classe=(outputs[0]["generated_text"][len(prompt):])

  return classe

GetEntity("Hi! My name is Matteo and i love football","Matteo")

In [ ]:
import pandas as pd
path_annotations="train_datasetV12.csv"
df = pd.read_csv(path_annotations, usecols=["Frase", "Name", "Start", "End", "Role", "Descriptor"])
df_protagonist = pd.DataFrame(columns=['frase','entity','sottoclassi'])

for index, row in df.iterrows():
  sub_classi=row['Descriptor'].split()
  entity = row['Name'].lower()
  sentence = row['Frase']
  descriptor=row['Descriptor']
  nuove_righe=[]

  if 'foreign' in sub_classi:
    sub_classi.remove('foreign')
    sub_classi.remove('adversary')
    sub_classi.append('Foreign Adversary')

  sub_classi = [f"{pr.capitalize()}" for pr in sub_classi]

  for sb in sub_classi:
    new_row = {
        'frase': sentence,
        'entity': entity,
        'sottoclassi':sb,
        'Start':row['Start'],
        'End':row['End'],
        'Role':row['Role']
      }
    nuove_righe.append(new_row)

  temp_df = pd.DataFrame(nuove_righe)

  df_protagonist = pd.concat([df_protagonist, temp_df], ignore_index=True)

print(len(df_protagonist))
df_protagonist.sottoclassi.value_counts()


In [ ]:
da_migliorare=['Saboteur','Bigot','Traitor','Scapegoat','Martyr','Forgotten','Spy']
for index, row in df_protagonist.iterrows():
  entity = row['entity'].lower()
  sentence = row['frase']
  des=row['sottoclassi']
  nuove_righe=[]
  if des in da_migliorare:
    nuova_frase=GetEntity(sentence,entity)
    new_row = {
        'Frase': nuova_frase,
        'Name': entity,
        'Start':row['Start'],
        'End':row['End'],
        'Role':row['Role'],
        'Descriptor':des
      }
    nuove_righe.append(new_row)


  temp_df = pd.DataFrame(nuove_righe)
  df = pd.concat([df, temp_df], ignore_index=True)
